In [1]:
!pip install pymorphy2

In [2]:
import os
import json
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
import nltk
nltk.download('punkt')
import re
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
!pip install corus

## 1. Толкования из файлика

Я собрала толкования из МАС и записала их в json с ключами `word` для слова и `meanings` для списка значений.

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
m_path = 'hw3_meanings.json'

In [6]:
os.getcwd()

'/content'

In [7]:
with open(os.path.join('/content/drive/MyDrive/', m_path),
          'r', encoding="utf-8") as f:
  data = json.load(f)
  print(data[0]['word'])

крона


In [8]:
words = dict()
for k in data:
    words[k['word']] = {'meanings':k['meanings'], 'texts':[]}

In [9]:
print(*words.keys())

крона крошка час голова таз


## 2. Контексты

Буду использовать корпуса с готовой разметкой, т.к. ручной лемматизатор грузился сто часов.

### НКРЯ

In [10]:
!wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/RNC_texts.rar

--2023-12-06 23:17:40--  https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/RNC_texts.rar
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dialogue-evaluation/morphoRuEval-2017/master/RNC_texts.rar [following]
--2023-12-06 23:17:40--  https://raw.githubusercontent.com/dialogue-evaluation/morphoRuEval-2017/master/RNC_texts.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5775644 (5.5M) [application/octet-stream]
Saving to: ‘RNC_texts.rar’

RNC_texts.rar       100%[===================>]   5.51M  --.-KB/s    in 0.08s   

2023-12-06 23:17:40 (69.4 MB/s) - ‘RNC_texts.rar’ saved [57

In [11]:
!unrar x RNC_texts.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from RNC_texts.rar


Would you like to replace the existing file RNCgoldInUD_Morpho.conll
75436890 bytes, modified on 2017-03-02 16:16
with a new one
75436890 bytes, modified on 2017-03-02 16:16

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit N

No files to extract


In [12]:
!rm RNC_texts.rar

In [13]:
from corus import load_morphoru_rnc

path = 'RNCgoldInUD_Morpho.conll'
records = load_morphoru_rnc(path)

In [14]:
next(records).tokens[0].lemma

'кстати'

In [15]:
for sent in records:
    for word in sent.tokens:
        if word.lemma in words.keys():
            words[word.lemma]['texts'].append(sent.tokens)

### ГИКРЯ

In [16]:
!wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/GIKRYA_texts_new.zip

--2023-12-06 23:21:38--  https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/GIKRYA_texts_new.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dialogue-evaluation/morphoRuEval-2017/master/GIKRYA_texts_new.zip [following]
--2023-12-06 23:21:38--  https://raw.githubusercontent.com/dialogue-evaluation/morphoRuEval-2017/master/GIKRYA_texts_new.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8961678 (8.5M) [application/zip]
Saving to: ‘GIKRYA_texts_new.zip’

GIKRYA_texts_new.zi 100%[===================>]   8.55M  --.-KB/s    in 0.09s   

2023-12-06 23:21:39 (99.8 MB/s) - ‘GIKRY

In [17]:
!unzip GIKRYA_texts_new.zip
!rm GIKRYA_texts_new.zip

Archive:  GIKRYA_texts_new.zip
replace gikrya_new_test.out? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace gikrya_new_train.out? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [18]:
from corus import load_morphoru_gicrya

path = 'gikrya_new_test.out'
records = load_morphoru_gicrya(path)

In [19]:
%timeit

for sent in records:
    for word in sent.tokens:
        if word.lemma in words.keys():
            words[word.lemma]['texts'].append(sent.tokens)

In [20]:
# проверяем для всего ли есть тексты

for w in words.keys():
  print(len(words[w]['texts']))

11
10
554
805
13


In [21]:
for k in words['крона']['texts']:
    text = [x.text for x in k]
    print(' '.join(text))

Они выше самых высоких каштанов , светло-зелёными холмами высятся их кроны над лесом .
В этом месте я сделал жест , указав на крону одного из камфорных деревьев , под которыми мы проходили .
На нас пахнуло холодом , зашелестели , зашептали густые листья в кронах деревьев .
Изредка солнечный свет пробивался сквозь кроны деревьев длинными тонкими пальцами , и тогда схваченные этими пальцами капли дождя сверкали , как хрустальные стёклышки .
Он развернул Андрея к окну , и тот увидел кроны деревьев , бешено проносящиеся мимо стекла слева направо .
Первыми забили тревогу питерское " Единство " и нижегородская " Крона ", которые , перебиваясь с воды на хлеб , с миру по нитке искали средства для поездки на очередной тур первенства .
Специально для национальных героев Центр репродукции установил льготный тариф -- 500 словацких крон вместо обычных 1750 .
Шишки пихты кавказской , подобно свечкам , украшают осенью самый верх кроны ; их никто не видел лежащими на земле , поскольку шишки у пихты ра

## 3. Классификация контекстов

In [22]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-tqnfushx
  Running command git clone --filter=blob:none --quiet https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-tqnfushx
  Resolved https://github.com/lopuhin/python-adagram.git to commit cf3639f10d6a1efbcb602f45a1da89ef55ce5794
  Preparing metadata (setup.py) ... done


In [23]:
import adagram
import numpy as np
import pandas as pd

In [24]:
# сначала причешем контексты

for w in words.keys():
     words[w]['texts'] = [list(map(lambda x: x.lemma, k)) for k in words[w]['texts']]

In [25]:
print(*words['крона']['texts'][0])

они высокий самый высокий каштан , светло-зеленый холм выситься их крона над лес .


In [26]:
!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > all.a010.p10.d300.w5.m100.nonorm.s

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394M  100 1394M    0     0  53.0M      0  0:00:26  0:00:26 --:--:-- 41.9M


In [27]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.s')

In [28]:
vm.word_sense_probs('голова')

[(0, 0.15136618792493253),
 (2, 0.18463972125993772),
 (3, 0.23746755179898937),
 (4, 0.031868947553770864),
 (5, 0.18276442527655343),
 (6, 0.21124967139097275)]

In [29]:
for w in words.keys():
    words[w]['adas'] = []
    for text in words[w]['texts']:
        means = vm.disambiguate(w, text)
        words[w]['adas'].append(np.argmax(means))

/usr/local/lib/python3.10/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [30]:
words['крошка']['adas']

[0, 0, 0, 1, 0, 0, 0, 2, 1, 0]

In [31]:
!pip install simple-elmo

In [32]:
!wget http://vectors.nlpl.eu/repository/20/196.zip

--2023-12-06 23:22:56--  http://vectors.nlpl.eu/repository/20/196.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206986351 (197M) [application/zip]
Saving to: ‘196.zip.1’

196.zip.1           100%[===================>] 197.40M  23.0MB/s    in 9.3s    

2023-12-06 23:23:06 (21.1 MB/s) - ‘196.zip.1’ saved [206986351/206986351]



In [33]:
from simple_elmo import ElmoModel
from sklearn.cluster import AffinityPropagation

model = ElmoModel()
model.load("196.zip")

/usr/local/lib/python3.10/dist-packages/simple_elmo/model.py:531: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  lstm_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(


'The model is now loaded.'

In [34]:
def get_elmo_vectors(word, tokens, model):
    all_vectors = model.get_elmo_vectors(tokens)
    word_vecs = []
    for i in range(len(tokens)):
        try:
            word_vecs.append(all_vectors[i][tokens[i].index(word)])
        except ValueError:  # если нормализация накосячила и лемму не найти
            continue
    return word_vecs

In [35]:
for w in words.keys():
    big_vecs = model.get_elmo_vectors(words[w]['texts'])
    vecs = []
    for i in range(len(words[w]['texts'])):
        vec = big_vecs[i][words[w]['texts'][i].index(w)]
        vecs.append(vec)

    vect = AffinityPropagation(random_state=0).fit(np.stack(vecs))
    words[w]['elmo'] = list(vect.labels_)

In [36]:
words['крона']['elmo']

[2, 3, 3, 3, 3, 0, 1, 2, 3, 3, 2]

## 4. Кластеризация словарных толкований

In [44]:
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
import nltk
from itertools import chain

In [45]:
#токенизатор из старой домашки + лемматизация

def clean_up(pp: str):
    morph = MorphAnalyzer()
    sent = sent_tokenize(pp.lower())
    tok = list(map(lambda x: word_tokenize(x), sent))
    tok = list(chain(*tok))
    depunct = list(filter(lambda x: x not in punctuation, tok))
    depunct = list(filter(lambda x: x not in '«»–', depunct))
    lem = list(map(lambda x: morph.parse(x)[0].normal_form, depunct))

    return lem

In [46]:
for w in words.keys():
    words[w]['adas_mas'] = []
    for text in words[w]['meanings']:
        means = vm.disambiguate(w, clean_up(text))
        words[w]['adas_mas'].append(np.argmax(means))

/usr/local/lib/python3.10/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [47]:
words['голова']['adas_mas']

[5, 3, 4, 4, 5, 5]

In [49]:
for w in words.keys():
    padded = [[w]+clean_up(x) for x in words[w]['meanings']]
    big_vecs = model.get_elmo_vectors(padded)
    vecs = []
    for i in range(len(words[w]['meanings'])):
        vec = big_vecs[i][padded[i].index(w)]
        vecs.append(vec)

    vect = AffinityPropagation(random_state=0).fit(np.stack(vecs))
    words[w]['elmo_mas'] = list(vect.labels_)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_affinity_propagation.py:53: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_affinity_propagation.py:53: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(


In [50]:
words['голова']['elmo_mas']

[1, 0, 0, 0, 0, 1]

## 5. Разметка

In [80]:
#вот так я размечала

for w in words.keys():
    for j in range(5, 10):
        print(words[w]['elmo_mas'])
        print(words[w]['adas_mas'])
        print(*words[w]['texts'][j])
        print(words[w]['elmo'][j])
        print(words[w]['adas'][j])
        break

words['крона']['rank_elma'] = [0, 1, 0, 0, 0]
words['крона']['rank_adas'] = [1, 1, 1, 1, 1]
words['крошка']['rank_elma'] = [0, 0, 0, 0, 0]
words['крошка']['rank_adas'] = [1, 1, 0, 0, 1]
words['час']['rank_elma'] = [1, 0, 0, 1, 0]
words['час']['rank_adas'] = [1, 0, 1, 0, 0]
words['голова']['rank_elma'] = [0, 0, 0, 0, 0]
words['голова']['rank_adas'] = [1, 1, 0, 0, 0]
words['таз']['rank_elma'] = [0, 1, 0, 0, 0]
words['таз']['rank_adas'] = [1, 0, 0, 1, 1]

[0, 1]
[0, 1]
первый забить тревога питерский " единство " и нижегородский " крона ", который , перебиваться с вода на хлеб , с мир по нитка искать средство для поездка на очередной тур первенство .
0
2
[0, 0, 1, 1]
[0, 0, 1, 0]
горячий солнечный свет падать на скатерть , покрыть липкий пятно и крошка , и Андрей вдруг подумать , что для миллион луч это настоящий трагедия -- начать свой путь на поверхность солнце , пронестись сквозь бесконечный пустота космос , пробить многокилометровый небо -- и всё только для то , чтобы угаснуть на отвратительный останки вчерашний суп .
1
0
[0, 1, 1, 1, 1, 1]
[4, 4, 4, 5, 4, 0]
два смена в детский сад подряд -- это 16 час .
0
4
[1, 0, 0, 0, 0, 1]
[5, 3, 4, 4, 5, 5]
я казаться , что кто надо , кто понимать , кто правильно расставлять приоритет , тот не прийти это даже в голова .
26
3
[0, 1]
[1, 0]
к то же снижать риск воспаление орган таз .
3
0


## 6. Accuracy and comparison

In [67]:
elma = []
adas = []
for w in words.keys():
    elma.extend(words[w]['rank_elma'])
    adas.extend(words[w]['rank_adas'])

print('Accuracy for ElmoModel {}%'.format(100 * sum(elma)/ len(elma)))
print('Accuracy for AdaGram {}%'.format(100 * sum(adas)/ len(adas)))

Accuracy for ElmoModel 16.0%
Accuracy for AdaGram 60.0%


`AdaGram` справляется сильно лучше, чем `ElmoModel`. Почему?

`AdaGram` считает вероятности появления определённого значения и у него есть внутренний порог для адекватного количества смыслов на слово. Для `ElmoModel` я просто кластеризовала контекстные вектора вручную. Хотя значения есть употребление, ближайший контекст всё-таки не полностью идентичен значению. СКлёрновский классификатор просто разделял очень непохожие контексты на очень разные значения, поэтому WSD с `ElmoModel` находил в текстах по 60+ разных значения, когда в словаре было только 6. Например:

In [77]:
print('Значение в тексте {}'.format(words['голова']['elmo'][7]))
print('Значения в словаре {}'.format(words['голова']['elmo_mas']))
print(*words['голова']['texts'][7])

Значение в тексте 71
Значения в словаре [1, 0, 0, 0, 0, 1]
леонид броневой сыграть мюллер , это — первый роль , принести в то время сорокапятилетний актер известность ( кстати , вот еще анекдот : « штирлиц выстрелить мюллер в голова , но она отлететь от он .
